In [1]:
import config
import tensorflow as tf
tf.app.flags.DEFINE_string('f', '', 'kernel')
from collections import deque
import model
from dataUtils import *
from logger import MyLogger
import sys
import PTB_data_reader
import time
import numpy as np
import lstm_char_cnn
import pickle
import dataloader
tf.logging.set_verbosity(tf.logging.ERROR)


logger = MyLogger("SentiMain")

W0901 18:42:15.496636 140377599600448 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.


In [2]:
# load twitter data
# load_data(FLAGS.data_file_path)
load_data_fast()

#load PTB data
# word_vocab, char_vocab, word_tensors, char_tensors, max_word_length = \
#     PTB_data_reader.load_data(FLAGS.data_dir, FLAGS.max_word_length, char_vocab, eos=FLAGS.EOS)
word_vocab, char_vocab, word_tensors, char_tensors = \
    PTB_data_reader.load_data_fast()
max_word_length = FLAGS.max_word_length
train_reader = PTB_data_reader.DataReader(word_tensors['train'], char_tensors['train'],
                          FLAGS.batch_size, FLAGS.max_sent_len) 

#load sentiment analysis data
sentiReader = dataloader.SentiDataLoader(
                                        dirpath = '/home/hadoop/trainingandtestdata',
                                        trainfile = 'training.1600000.processed.noemoticon.csv', 
                                        testfile = 'testdata.manual.2009.06.14.csv', 
                                        charVocab = char_vocab
                        )
# sentiReader.load_data()
sentiReader.load_data_fast(
                        '/home/hadoop/ERD/data/senti_train_data.pickle',
                        '/home/hadoop/ERD/data/senti_train_label.pickle',
                        '/home/hadoop/ERD/data/senti_test_data.pickle',
                        '/home/hadoop/ERD/data/senti_test_label.pickle'
                          )


# (self, input_dim, hidden_dim, max_seq_len, max_word_num, class_num, action_num):
print(  FLAGS.embedding_dim, FLAGS.hidden_dim, 
            FLAGS.max_seq_len, FLAGS.max_sent_len, 
                FLAGS.class_num, FLAGS.action_num   )
logger.info(    (FLAGS.embedding_dim, FLAGS.hidden_dim, 
                    FLAGS.max_seq_len, FLAGS.max_sent_len, 
                        FLAGS.class_num, FLAGS.action_num)  )

print(get_curtime() + " Data loaded.")
logger.info(get_curtime() + " Data loaded.")

max_sent: 31 ,  max_seq_len: 101
5802 data loaded


I0901 18:42:31.175283 140377599600448 logger.py:24] (300, 64, 101, 31, 2, 2)
I0901 18:42:31.176598 140377599600448 logger.py:24] 2019-09-01 18:42:31 Data loaded.


300 64 101 31 2 2
2019-09-01 18:42:31 Data loaded.


In [3]:
# # save the Twitter data
# data = get_data()
# with open('data/data_dict.txt', 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # save the PTB data
# with open('data/char_tensors.txt', 'wb') as handle:
#     pickle.dump(char_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_tensors.txt', 'wb') as handle:
#     pickle.dump(word_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/char_vocab.txt', 'wb') as handle:
#     pickle.dump(char_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_vocab.txt', 'wb') as handle:
#     pickle.dump(word_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save the senti data
# with open('data/senti_train_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_train_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_label, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/senti_test_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_test_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_label, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [15]:
from tensorflow.losses import Reduction
from model import adict

def InferSentiTrainGraph(char_model, lm, senti_model, 
                            batchsize, max_word_num, max_char_num, 
                            embedding_dim, hidden_dim, sent_num, learning_rate):
    sent_x = tf.placeholder(tf.int32, shape = [batchsize, max_word_num, max_char_num])
    sent_y = tf.placeholder(tf.float32, shape = [batchsize, sent_num])
    words2vec = char_model(sent_x) #[None, max_word_num, kernerl_size]
    cnn_outs = tf.reshape(words2vec, [batchsize, max_word_num, sum(char_model.kernel_features)])
    
    cnn_outs_list = [tf.squeeze(x, [1]) 
    for x in tf.split(cnn_outs, max_word_num, 1)]
    lm_init_state = lm.cell.zero_state(
                            batchsize, 
                            dtype=tf.float32
                        )
    words_embedding, sentence_embedding = tf.contrib.rnn.static_rnn(
                                        lm.cell, 
                                        cnn_outs_list,
                                        initial_state=lm_init_state, 
                                        dtype=tf.float32
                                    )     
    
    words_embedding = tf.identity(words_embedding, "rnn_out_puts")
    words_embedding = tf.transpose(words_embedding, [1, 0, 2])
    words_feature = tf.math.reduce_max( words_embedding , axis=1)
    print("words_feature:", words_feature)
    print("sentence_embedding:", sentence_embedding)
    senti_features =  sentence_embedding[-1][-1] + words_feature 
    print("senti_features:", senti_features)
#     senti_features = senti_model(words_embedding)
    classifier = tf.layers.Dense(sent_num, activation=tf.nn.relu, trainable=True)
    senti_rst = classifier(senti_features)
    sent_scores = tf.nn.softmax(senti_rst, axis=1)
    sent_pred = tf.argmax(sent_scores, 1, name="predictions")
#     sent_loss = tf.losses.softmax_cross_entropy(
#                         sent_y,
#                         senti_rst,
#                         weights=1.0,
#                         label_smoothing=0,
#                         scope=None,
#                         loss_collection=tf.GraphKeys.LOSSES,
#                         reduction=Reduction.SUM_BY_NONZERO_WEIGHTS
#                     )
    sent_loss = tf.reduce_mean((sent_y-senti_rst)*(sent_y-senti_rst))
    sent_correct_predictions = tf.equal(sent_pred, tf.argmax(sent_y, 1))
    sent_acc = tf.reduce_mean(tf.cast(sent_correct_predictions, "float"), name="accuracy")
    sent_global_step = tf.Variable(0, name="global_step", trainable=False)
    sent_train_op = tf.train.AdagradOptimizer(0.1).minimize(sent_loss, sent_global_step)        
    return adict(
                # dropout_keep_prob = self.dropout_keep_prob,
                sent_x = sent_x,
                sent_y = sent_y,
                drop_out = lm.drop_out,
                initial_rnn_state = lm_init_state, 
                feature = senti_features,
                sent_scores = sent_scores,
                sent_pred = sent_pred,
                sent_loss = sent_loss,
                sent_acc = sent_acc,
                sent_global_step = sent_global_step,
                sent_train_op = sent_train_op
            )

In [9]:
def TrainSentiModel(sess, saver, logger, summary_writer, model_dir, train_model, senti_reader, train_batch, test_batch):
    train_iter = 100000
    rnn_state = sess.run(train_model.initial_rnn_state)
    for t_epoch in range(10): 
        # for validation
        sum_acc = 0.0
        sum_loss = 0.0
        for t_iter in range(train_iter):
            data_X, data_Y = senti_reader.GetTrainingBatch(
                                            t_iter, 
                                            train_batch
                            )
            feed_dic = {
                        train_model.sent_x: data_X, 
                        train_model.drop_out: 0.8,
                        train_model.sent_y: data_Y,
                        train_model.initial_rnn_state:rnn_state
            }
            _, step, loss, acc = sess.run(
                                        [train_model.sent_train_op, 
                                         train_model.sent_global_step, 
                                         train_model.sent_loss, 
                                         train_model.sent_acc], 
                                        feed_dic)
            summary = tf.Summary(value=[
                tf.Summary.Value(tag="step_train_loss", simple_value=loss),
                tf.Summary.Value(tag="step_train_acc", simple_value=acc),
            ])
            summary_writer.add_summary(summary, step)
            
            sum_loss += loss
            sum_acc += acc
            if t_iter % 10 == 9:
                sum_loss = sum_loss / 10
                sum_acc = sum_acc / 10
                ret_acc = sum_acc
                print(get_curtime() + " Step: " + str(step) + " Training loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
                logger.info(get_curtime() + " Step: " + str(step) + " Training loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
                sum_acc = 0.0
                sum_loss = 0.0
        saver.save(sess, model_dir)
        # for validation
        sum_acc = 0.0
        sum_loss = 0.0
        for t_iter in range(100):
            data_X, data_Y = senti_reader.GetTestData(t_iter, test_batch)
            feed_dic = {
                        train_model.sent_x: data_X, 
                        train_model.drop_out: 1.0,
                        train_model.sent_y: data_Y,
                        train_model.initial_rnn_state:rnn_state
            }
            loss, acc = sess.run([train_model.sent_loss, train_model.sent_acc], feed_dic)
            sum_loss += loss
            sum_acc += acc    
        sum_loss = sum_loss / 100
        sum_acc = sum_acc / 100
        ret_acc = sum_acc
        print(get_curtime() + " Step: " + str(step) + " validation loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
        logger.info(get_curtime() + " Step: " + str(step) + " validation loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
        
        summary = tf.Summary(value=[
                tf.Summary.Value(tag="step_valid_loss", simple_value=sum_loss),
                tf.Summary.Value(tag="step_valid_acc", simple_value=sum_acc),
            ])
        summary_writer.add_summary(summary, t_epoch)
        
        sum_acc = 0.0
        sum_loss = 0.0

        
    
        

In [10]:
import importlib

In [11]:
importlib.reload(model)

<module 'model' from '/home/hadoop/ERD/model.py'>

In [17]:
# reuse model to train senti model
gpu_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.4
device = '/GPU:0'
with tf.Graph().as_default() as g:
    with tf.Session(graph=g, config=gpu_config) as sess:
        w2v = lstm_char_cnn.WordEmbedding(
                        max_word_length = FLAGS.max_char_num , 
                        char_vocab_size = char_vocab.size, 
                        char_embed_size = FLAGS.char_embed_size, 
                        kernels = eval(FLAGS.kernels), 
                        kernel_features = eval(FLAGS.kernel_features), 
                        num_highway_layers = FLAGS.highway_layers,
                        embedding_dim = FLAGS.embedding_dim
                    )
        lstm_lm = lstm_char_cnn.LSTM_LM(
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.max_sent_len, 
                    rnn_size = FLAGS.embedding_dim, 
                    num_rnn_layers = FLAGS.rnn_layers, 
                    word_vocab_size = word_vocab.size
                )

        char_train_graph = lstm_char_cnn.infer_train_model(
                            w2v, lstm_lm, 
                            batch_size = FLAGS.batch_size, 
                            num_unroll_steps = FLAGS.max_sent_len, 
                            max_word_length = FLAGS.max_char_num, 
                            learning_rate = FLAGS.learning_rate,
                            max_grad_norm = FLAGS.max_grad_norm
                         )
        val_list1 = tf.global_variables()
        saver = tf.train.Saver(val_list1, max_to_keep=4)
        sess.run(tf.variables_initializer(val_list1))
        checkpoint = tf.train.get_checkpoint_state("lstmCharCNNGPU/")
        if checkpoint and checkpoint.model_checkpoint_path:
            saver.restore(sess, checkpoint.model_checkpoint_path)
            
        with tf.device(device):
            #sentiment analysis model
            s_model = model.SentiModel(FLAGS.hidden_dim, 5)
            senti_train_graph = InferSentiTrainGraph(
                                    w2v, 
                                    lstm_lm, 
                                    s_model, 
                                    batchsize=20,
                                    max_word_num = sentiReader.max_sent_len, 
                                    max_char_num = FLAGS.max_char_num, 
                                    hidden_dim = FLAGS.hidden_dim, 
                                    sent_num = FLAGS.sent_num,
                                    embedding_dim = FLAGS.embedding_dim,
                                    learning_rate = 0.1
                                )
        val_list2 = tf.global_variables()
        saver2 = tf.train.Saver(val_list2, max_to_keep=4)
        uninitialized_vars = list( filter(lambda var: var not in val_list1, val_list2) )
        sess.run(tf.variables_initializer(uninitialized_vars))
        summary_writer = tf.summary.FileWriter("SentiCNNGPU/", graph=sess.graph)
        TrainSentiModel(sess, saver, logger, summary_writer, "SentiCNNGPU/", senti_train_graph, sentiReader, 20, 20)
        

input_: Tensor("input:0", shape=(20, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(620, 1100), dtype=float32)
input_: Tensor("Placeholder_1:0", shape=(20, 41, 21), dtype=int32, device=/device:GPU:0)
input_cnn: Tensor("Embedding_2/CNN_OUT/add_7:0", shape=(820, 1100), dtype=float32, device=/device:GPU:0)
words_feature: Tensor("Max:0", shape=(20, 300), dtype=float32, device=/device:GPU:0)
sentence_embedding: (LSTMStateTuple(c=<tf.Tensor 'LSTM_1/rnn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/Add_143:0' shape=(20, 300) dtype=float32>, h=<tf.Tensor 'LSTM_1/rnn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/Mul_215:0' shape=(20, 300) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'LSTM_1/rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Add_143:0' shape=(20, 300) dtype=float32>, h=<tf.Tensor 'LSTM_1/rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_215:0' shape=(20, 300) dtype=float32>))
senti_features: Tensor("add:0", shape=(20, 300), dtype=float32, device=/device:GPU:0)


I0901 10:18:55.632224 140123195463488 logger.py:24] 2019-09-01 10:18:55 Step: 10 Training loss: 1.0986123085021973 accuracy: 0.5049999982118607


2019-09-01 10:18:55 Step: 10 Training loss: 1.0986123085021973 accuracy: 0.5049999982118607


I0901 10:18:56.046760 140123195463488 logger.py:24] 2019-09-01 10:18:56 Step: 20 Training loss: 1.1245155692100526 accuracy: 0.5150000095367432


2019-09-01 10:18:56 Step: 20 Training loss: 1.1245155692100526 accuracy: 0.5150000095367432


...

I0901 10:19:03.311193 140123195463488 logger.py:24] 2019-09-01 10:19:03 Step: 200 Training loss: 1.0986123085021973 accuracy: 0.4850000023841858


2019-09-01 10:19:03 Step: 200 Training loss: 1.0986123085021973 accuracy: 0.4850000023841858


I0901 10:19:15.429938 140123195463488 logger.py:24] 2019-09-01 10:19:15 Step: 210 Training loss: 1.0986123085021973 accuracy: 0.4299999997019768


2019-09-01 10:19:15 Step: 210 Training loss: 1.0986123085021973 accuracy: 0.4299999997019768


KeyboardInterrupt: 